Baby AGI with agent notes prepared for [CAS ETH in ML in Finance and Insurance](https://finsuretech.ethz.ch/continuing-education/cas-ml-in-finance-and-insurance.html) by Eleni Verteouri.

# BabyAGI with Tools

This notebook builds on top of [baby agi](https://github.com/yoheinakajima/babyagi/tree/main), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

In [1]:
!pip install langchain

In [2]:
!pip install langchain_experimental

In [3]:
!pip install langchain_openai

## Install and Import Required Modules

In [4]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [5]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [6]:
import os

OPENAI_API_TOKEN = "your key"
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [7]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

Please create free SERP API key here: https://serpapi.com/

In [8]:
SERP_API_TOKEN="your key"
os.environ["SERPAPI_API_KEY"] = SERP_API_TOKEN

In [9]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [10]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [11]:
OBJECTIVE = "Write a weather report for Zurich, Switzerland today"

In [12]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [13]:
baby_agi({"objective": OBJECTIVE})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a list of tasks to complete
Action: TODO
Action Input: Create a todo list for today's weather report in Zurich, Switzerland
Observation: 

1. Check the weather forecast for Zurich, Switzerland.
2. Gather necessary materials such as a pen, paper, and a computer.
3. Research current weather patterns and trends in Zurich.
4. Determine the expected temperature, precipitation, and wind conditions for the day.
5. Make note of any potential weather hazards or warnings.
6. Create a template for the weather report, including sections for current conditions, hourly forecast, and extended forecast.
7. Begin writing the report, starting with a brief overview of the current weather.
8. Use visuals such as charts or graphs to illustrate the forecasted conditions.
9. Include any relevant information or events that may impact the weather, such as a local 

{'objective': 'Write a weather report for Zurich, Switzerland today'}